In [1]:
import json
import os
import re
import time

from msgspec.json import decode
from msgspec import Struct
from openai import OpenAI
from tqdm import tqdm
import numpy as np

In [2]:
class NewsItem(Struct):
    title: str
    source: str
    text: str
    bias: str

with open("BIGNEWSALIGN_formatted.json", "rb") as f:
    data = decode(f.read(), type=list[list[NewsItem]])

In [3]:
folder_path = './evaluations/gemma-7b/finetune/q5_k_m'
client = OpenAI(base_url="https://chester-spirit-int-strand.trycloudflare.com/v1", api_key="lm-studio")

In [4]:
selected_indexes = [1549, 1220, 2664, 693, 819, 990, 872, 727, 1316, 637, 893, 725, 602, 426, 567, 535, 576, 1197, 898, 969, 613, 972, 803, 1942, 853, 938, 784, 579, 922, 676, 1523, 1442, 1715, 1547, 1776, 1594, 1567, 1384, 1060, 2271, 1770, 1815, 1014, 2446, 1004, 1602, 1462, 1536, 1738, 2201, 1677, 1843, 1115, 2123, 1955, 1236, 1629, 1374, 1799, 2096, 1444, 1333, 1588, 1685, 2324, 1847, 1674, 1449, 1711, 1366, 2048, 2252, 2034, 1841, 2005, 1635, 2336, 2111, 2183, 1802, 2369, 1508, 1687, 1643, 1801, 1389, 1468, 2189, 1911, 1498, 1704, 1872, 1928, 1646, 1608, 2128, 2003, 2387, 1551, 1469]

In [5]:
instruction = """
This is an overall sentiment reasoning task for a news article. You will be given a INPUT news article.
First, find the BIAS of the news article (i.e., "left", "lean left", "center", "lean right", "right".)
Second, present CLUES (i.e., keywords, phrases, contextual information, semantic relations, semantic meaning, tones, references) that support the sentiment determination of input.
Finally, deduce a diagnostic REASONING process from premises (i.e., bias, clues, input) that supports the sentiment determination.
"""

In [6]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:

{}

### Response:
{}"""

In [7]:
for i in tqdm(selected_indexes):
    news_items = data[i]
    os.makedirs(f"{folder_path}/{i}")
    for k, news_item in enumerate(news_items):
        content = alpaca_prompt.format(
                instruction, 
                f'{news_item.title}\n\n{news_item.text}', 
                "")
        if i == 1549:
            print(content)
        completion = client.chat.completions.create(
            model="",
            messages=[{"role": "user","content": content}],
            temperature=0.8,
            max_tokens=512
        )
        with open(f"{folder_path}/{i}/{k}.txt", "w", encoding="utf-8") as _temp_file:
            _temp_file.write(completion.choices[0].message.content)

100%|██████████| 98/98 [22:03<00:00, 13.50s/it]


In [6]:
#Predicted/Generated responses

# Initialize counters
total_processed = 0
predicted_biases = []
exception_files = []

# Define the regex pattern to find the bias
pattern = r"(BIAS|Bias) - (LEFT|LEAN LEFT|CENTER|LEAN RIGHT|RIGHT|Left|Lean Left|Center|Lean Right|Right)"

# Process files from 100 to 499
for index in selected_indexes:
    # file_path = os.path.join(folder_path, f'{file_number}.txt')
    index_folder_path = os.path.join(folder_path, str(index))
    
    for file_number in tqdm(os.listdir(index_folder_path)):
        file_path = os.path.join(index_folder_path, f"{file_number}")
        try:
            # Open and read the file
            with open(file_path, 'r', encoding="utf-8") as _file:
                content = _file.read()
                match = re.search(pattern, content)
                # Get only the specific bias part of the matched group
                bias_value = match.group(2) if match else "Bias not found"
                if bias_value == "Bias not found":
                    print(f"file path - {file_path} - {bias_value}")
                predicted_biases.append(bias_value.upper())

        except Exception as e:
            # Keep track of files where an exception occurred
            print(e)
            exception_files.append(file_number)


# Output the results
print(f'Total files processed: {len(predicted_biases)}')
print(f'Files with exceptions: {exception_files}')
print(predicted_biases)

100%|██████████| 5/5 [00:00<?, ?it/s]

Total files processed: 500
Files with exceptions: []
['LEFT', 'CENTER', 'LEFT', 'LEAN RIGHT', 'LEAN RIGHT', 'LEAN RIGHT', 'LEAN RIGHT', 'CENTER', 'LEAN RIGHT', 'RIGHT', 'CENTER', 'CENTER', 'CENTER', 'LEAN RIGHT', 'LEAN RIGHT', 'LEFT', 'CENTER', 'RIGHT', 'LEAN RIGHT', 'RIGHT', 'LEFT', 'LEAN LEFT', 'LEAN LEFT', 'LEAN RIGHT', 'LEAN LEFT', 'LEFT', 'LEAN RIGHT', 'CENTER', 'LEAN RIGHT', 'LEAN RIGHT', 'LEFT', 'LEAN LEFT', 'CENTER', 'LEAN RIGHT', 'LEAN RIGHT', 'LEFT', 'LEAN LEFT', 'CENTER', 'CENTER', 'LEAN RIGHT', 'LEAN LEFT', 'CENTER', 'CENTER', 'LEAN RIGHT', 'RIGHT', 'LEFT', 'LEAN LEFT', 'CENTER', 'CENTER', 'CENTER', 'LEFT', 'LEAN LEFT', 'LEAN LEFT', 'LEAN RIGHT', 'RIGHT', 'LEFT', 'LEFT', 'RIGHT', 'CENTER', 'RIGHT', 'LEFT', 'LEAN LEFT', 'LEAN LEFT', 'LEAN LEFT', 'LEAN RIGHT', 'LEFT', 'LEAN LEFT', 'LEAN LEFT', 'LEAN RIGHT', 'RIGHT', 'LEFT', 'LEFT', 'CENTER', 'LEAN RIGHT', 'LEAN RIGHT', 'LEFT', 'LEFT', 'CENTER', 'RIGHT', 'LEAN RIGHT', 'LEFT', 'LEAN LEFT', 'CENTER', 'LEAN RIGHT', 'RIGHT', 'LEAN

In [7]:
actual_biases = []
for index in selected_indexes:
    news_items = data[index]
    for news_item in news_items:
        actual_biases.append(news_item.bias.upper())

print(len(actual_biases))
actual_biases

500


['LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT',
 'LEFT',
 'LEAN LEFT',
 'CENTER',
 'LEAN RIGHT',
 'RIGHT

In [8]:
def calculate_accuracy(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must be of the same length.")
    
    total_elements = len(list1)
    correct_matches = sum(1 for x, y in zip(list1, list2) if x == y)
    
    accuracy = correct_matches / total_elements * 100
    return accuracy

accuracy = calculate_accuracy(actual_biases, predicted_biases)
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 51.20%


In [9]:
def calculate_accuracy(actual_biases, predicted_biases):
    if len(actual_biases) != len(predicted_biases):
        raise ValueError("Lists must be of the same length.")
    
    total_elements = len(actual_biases)
    correct_matches = 0
    
    for actual, predicted in zip(actual_biases, predicted_biases):
        if actual == predicted:
            correct_matches += 1
        elif actual == "LEFT" and predicted == "LEAN LEFT":
            correct_matches += 1
        elif actual == "LEAN LEFT" and predicted == "LEFT":
            correct_matches += 1
        elif actual == "RIGHT" and predicted == "LEAN RIGHT":
            correct_matches += 1
        elif actual == "LEAN RIGHT" and predicted == "RIGHT":
            correct_matches += 1
    
    accuracy = correct_matches / total_elements * 100
    return accuracy

accuracy = calculate_accuracy(actual_biases, predicted_biases)
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 65.00%


In [10]:
def calculate_distance(actual, predicted):
    bias_values = {"LEFT": 1, "LEAN LEFT": 2, "CENTER": 3, "LEAN RIGHT": 4, "RIGHT": 5}
    return abs(bias_values[actual] - bias_values[predicted])

def calculate_accuracy_and_distance(actual_biases, predicted_biases):
    if len(actual_biases) != len(predicted_biases):
        raise ValueError("Lists must be of the same length.")
    
    total_elements = len(actual_biases)
    correct_matches = 0
    total_distance = 0
    
    for actual, predicted in zip(actual_biases, predicted_biases):
        if actual == predicted:
            correct_matches += 1
        elif actual == "LEFT" and predicted == "LEAN LEFT":
            correct_matches += 1
            total_distance += 1
        elif actual == "LEAN LEFT" and predicted == "LEFT":
            correct_matches += 1
            total_distance += 1
        elif actual == "RIGHT" and predicted == "LEAN RIGHT":
            correct_matches += 1
            total_distance += 1
        elif actual == "LEAN RIGHT" and predicted == "RIGHT":
            correct_matches += 1
            total_distance += 1
        else:
            total_distance += calculate_distance(actual, predicted)
    
    accuracy = correct_matches / total_elements * 100
    avg_distance = total_distance / total_elements
    return accuracy, avg_distance


accuracy, avg_distance = calculate_accuracy_and_distance(actual_biases, predicted_biases)
print(f"Accuracy: {accuracy:.2f}%")
print(f"Average distance from correct bias: {avg_distance:.2f}")


Accuracy: 65.00%
Average distance from correct bias: 0.77


In [11]:
from sklearn.metrics import f1_score

# Calculating F1 score for each class
f1 = f1_score(actual_biases, predicted_biases, labels=['LEFT', 'LEAN LEFT', 'CENTER', 'LEAN RIGHT', 'RIGHT'], average=None)

# Printing the F1 scores for each class
print("F1 scores for each class:")
print("LEFT: {:.2f}".format(f1[0]))
print("LEAN LEFT: {:.2f}".format(f1[1]))
print("CENTER: {:.2f}".format(f1[2]))
print("LEAN RIGHT: {:.2f}".format(f1[3]))
print("RIGHT: {:.2f}".format(f1[4]))

F1 scores for each class:
LEFT: 0.62
LEAN LEFT: 0.43
CENTER: 0.40
LEAN RIGHT: 0.65
RIGHT: 0.49


In [12]:
# Calculating F1 micro score
f1 = f1_score(actual_biases, predicted_biases, labels=['LEFT', 'LEAN LEFT', 'CENTER', 'LEAN RIGHT', 'RIGHT'], average="micro")

print(f"F1 micro - {f1*100:.2f}%")

F1 micro - 51.20%


In [13]:
# Calculating F1 macro score
f1 = f1_score(actual_biases, predicted_biases, labels=['LEFT', 'LEAN LEFT', 'CENTER', 'LEAN RIGHT', 'RIGHT'], average="macro")

print(f"F1 macro - {f1*100:.2f}%")

F1 macro - 51.77%


In [14]:
from sklearn.metrics import f1_score

def merge_biases(biases):
    merged = []
    for bias in biases:
        if bias in ['LEFT', 'LEAN LEFT']:
            merged.append('LEFT')  # Merging 'LEFT' and 'LEAN LEFT'
        elif bias in ['RIGHT', 'LEAN RIGHT']:
            merged.append('RIGHT')  # Merging 'RIGHT' and 'LEAN RIGHT'
        else:
            merged.append(bias)  # 'CENTER' remains the same
    return merged

# Preprocess to merge biases
actual_merged = merge_biases(actual_biases)
predicted_merged = merge_biases(predicted_biases)

# Calculating F1 score for each class with merged categories
f1 = f1_score(actual_merged, predicted_merged, labels=['LEFT', 'CENTER', 'RIGHT'], average=None)

# Printing the F1 scores for each class
print("F1 scores for each class after merging categories:")
print("LEFT: {:.2f}".format(f1[0]))  # Includes both 'LEFT' and 'LEAN LEFT'
print("CENTER: {:.2f}".format(f1[1]))
print("RIGHT: {:.2f}".format(f1[2]))  # Includes both 'RIGHT' and 'LEAN RIGHT'

F1 scores for each class after merging categories:
LEFT: 0.66
CENTER: 0.40
RIGHT: 0.77


In [15]:
f1 = f1_score(actual_merged, predicted_merged, labels=['LEFT', 'CENTER', 'RIGHT'], average='micro')
print(f"F1 micro - {f1*100:.2f}%")

F1 micro - 65.00%


In [16]:
f1 = f1_score(actual_merged, predicted_merged, labels=['LEFT', 'CENTER', 'RIGHT'], average='macro')
print(f"F1 macro - {f1*100:.2f}%")

F1 macro - 61.10%
